In [ ]:
# Get URS creds and add to .netrc so we can do stuff!!
from getpass import getpass

NEDusr = 'URS name'
NEDpass = getpass()

with open('/home/jovyan/.netrc', 'w') as f:
    f.write('machine urs.earthdata.nasa.gov login ' + NEDusr + ' password ' + NEDpass + '\n')

In [ ]:
# Login into the Hyp3 API
from asf_hyp3 import API
api = API(NEDusr)
api.login(NEDpass)

In [ ]:
# Get HyP3 Sub Name
subs = api.get_subscriptions()
print(subs)
print(f"\nSubscription id: {subs[0]['id']}")

In [ ]:
# Look at products in subscription and print out download urls
products = api.get_products(sub_id=1444)
print(products)

print(f"\nURLs of products: \n")
for p in products:
    print(f"{p['url']}\n")

In [ ]:
# Download products from subscription.
!mkdir -p granules

import re
for p in products:
    url = p['url']
    _match = re.match(r'https://hyp3-download.asf.alaska.edu/asf/data/(.*).zip', url)
    granule = _match.group(1)
    
    print(f"Downloading '{granule}' from '{url}'")
    
    # Guess we need to download and unzip
    #!wget -O granules/{granule}.zip "{url}"
    #!unzip -d granules granules/{granule}.zip
    #!rm granules/{granule}.zip
    
    # This should work but doesn't...
    # See what's inside a remote granule. This requires some heavy memory
    # !gdalinfo /vsizip/vsicurl/{url}

In [ ]:
# Grab the paths of the VV
tiff_paths = !ls granules/*/*_VV.tif | sort
print(f"Tiff paths: {tiff_paths}")

In [ ]:
# Using Google Maps, get the rough bounding box for the subset
ulx = -149.1
lrx = -149.0
lry = 61.5
uly = 61.6
!echo {ulx} {lrx} {lry} {uly}

In [ ]:
# Cycle through and subset the tiffs in the products
!mkdir -p tiffs
for tiff_path in tiff_paths:
    _, granule_name, tiff_name = tiff_path.split('/')
    
    # Using the GDAL subset service, get a small subset around the Butte
    #!wget -O {granule_name}_VV.tiff "https://services.asf.alaska.edu/geospatial/subset?ulx={ulx}&lrx={lrx}&lry={lry}&uly={uly}&product={granule_name}.zip/{granule_name}/{tiff_name}"

    # GDAL service is out of service. Pretend that it isn't when calling the following equivalent command
    gdal_command = f"gdal_translate -projwin {ulx} {uly} {lrx} {lry} -projwin_srs 'WGS84' -co \"COMPRESS=DEFLATE\" -co \"TILED=YES\" -co \"COPY_SRC_OVERVIEWS=YES\" {tiff_path} tiffs/{granule_name}_VV.tiff"
    print(f"Calling the command: {gdal_command}")
    !{gdal_command}


In [ ]:
# Get info on the last subsetted tiff
!gdalinfo tiffs/{granule_name}_VV.tiff

In [ ]:
# Create the VRT for the downloaded subset geotiffs
# Grab all tiffs in the directory
!gdalbuildvrt -separate butte.vrt tiffs/*.tiff

In [ ]:
# We need dates for
!ls granules/*/*_VV.tif | sort | cut -c 27-41 > butte.dates
!cat butte.dates

### The following cells from here on out are largely taken from Lab 4 of Franz's class...

In [ ]:
# Set some paths
datadirectory='/home/jovyan/'
datefile='butte.dates'
imagefile='butte.vrt'

In [ ]:
# Import some modules
import pandas as pd
import gdal
import numpy as np
import time,os, glob

# Setup plotting inside the notebook
%matplotlib inline
import matplotlib.pylab as plt

In [ ]:
# Get some indices for plotting
dates=open(datefile).readlines()
tindex=pd.DatetimeIndex(dates)

In [ ]:
# Bands and times
j=1
print('Bands and dates for',imagefile)
for i in tindex:
    print("{:4d} {}".format(j, i.date()),end=' ')
    j+=1
    if j%5==1: print()

In [ ]:
# Open virtual dataset
img=gdal.Open(imagefile)

In [ ]:
print(img.RasterCount) # Number of Bands
print(img.RasterXSize) # Number of Pixels
print(img.RasterYSize) # Number of Lines

In [ ]:
# Read in raster data for the first two bands
raster_1 = img.GetRasterBand(1).ReadAsArray()
raster_2 = img.GetRasterBand(2).ReadAsArray()

In [ ]:
# Plot some things
fig = plt.figure(figsize=(18,10)) # Initialize figure with a size
ax1 = fig.add_subplot(221)  # 121 determines: 2 rows, 2 plots, first plot
ax2 = fig.add_subplot(222)  # 122 determines: 2 rows, 2 plots, second plot
ax3 = fig.add_subplot(223)  # 223 determines: 2 rows, 2 plots, third plot
ax4 = fig.add_subplot(224)  # 224 determines: 2 rows, 2 plots, fourth plot

# First plot: Image
bandnbr=1
ax1.imshow(raster_1,cmap='gray',vmin=-0.75,vmax=0.75) #,vmin=2000,vmax=10000)
ax1.set_title('Image Band {} {}'.format(bandnbr, tindex[bandnbr-1].date()))

# Second plot: Histogram
# IMPORTANT: To get a histogram, we first need to *flatten* 
# the two-dimensional image into a one-dimensional vector.
h = ax2.hist(raster_1.flatten(),bins=200,range=(0,1))
ax2.xaxis.set_label_text('Amplitude? (Uncalibrated DN Values)')
ax2.set_title('Histogram Band {} {}'.format(bandnbr, tindex[bandnbr-1].date()))


# Third plot: Image
bandnbr=2
ax3.imshow(raster_2,cmap='gray',vmin=-1,vmax=1) #,vmin=2000,vmax=10000)
ax3.set_title('Image Band {} {}'.format(bandnbr, tindex[bandnbr-1].date()))

# Fourth plot: Histogram
# IMPORTANT: To get a histogram, we first need to *flatten* 
# the two-dimensional image into a one-dimensional vector.
h = ax4.hist(raster_2.flatten(),bins=200,range=(0,1))
ax4.xaxis.set_label_text('Amplitude? (Uncalibrated DN Values)')
ax4.set_title('Histogram Band {} {}'.format(bandnbr, tindex[bandnbr-1].date()))